# Add Transfer Functions to an MTH5

This examples demonstrates how transfer functions can be added to an MTH5 file.  The transfer functions are stored at the `Station` level in a group called `TransferFunctionsGroup`.  Within this group station transfer functions are stored as a single group for each transfer function `TransferFunctionGroup`.  Each `TransferFunctionGroup` has a data set for each statistical estimate provided within the transfer function.  Currently supported estimates are

| Estimate           | Description                  |  Shape | Data Type |
|--------------------|------------------------------|--------|---------- |
| transfer_function  | Full transfer function       | n_periods x n_inputs x n_outputs | complex |
| transfer_function_error  | Full transfer function error estimation      | n_periods x n_inputs x n_outputs | real |
| impedance  | Only horizontal components, traditional impedance tensor       | n_periods x 2 x 2 |complex |
| impedance_error  | Only horizontal components, traditional impedance tensor error       | n_periods x 2 x 2 |real |
| tipper  | horizontal and vertical magnetic transfer function, Tipper  | n_periods x 1 x 2 | complex |
| tipper_error  | horizontal and vertical magnetic transfer function, Tipper error  | n_periods x 1 x 2 |real |
| inverse_signal_power  | covariance of input channels (sources)  | n_periods x n_inputs x n_inputs |complex |
| residual_covariance  | covariance of output channels (responses) | n_periods x n_outputs x n_outputs |complex |

**Note:** There are plans to add phase tensor and resitivity/phase estimations in the future.



# TF object

The TF object comes from `mt_metadata.transfer_functions.core.TF` and is meant to be the common container for transfer functions.  It has readers for:

* EDI
* EMFT XML
* Z-files (EMTF output)
* J-files (BIRRP output)
* AVG files (Zonge output)

The `TF` object has two important metadata objects `survey_metadata` and `station_metadata`.  Metadata from the other files are translated into these containers and translated back when writing a file.  

The statistical estimates are stored as `xarray.Datasets` that have coordinates of `period`, `input_channels`, `output_channels`.  This way the transfer function can be generalized.  `impedance` and `tipper` are stored in `transfer_function` and `TF` provides convenience functions to access `impedance` and `tipper` and associated errors.  Variances are stored as covariances for input channels (`inverse_signal_power`) and output channels (`residual_covariance`) when possible, and the `transfer_function_error` is stored as well.  

**Note:** There are future plans to include phase tensor and resistivity/phase representation as well.  

In [1]:
from mth5.mth5 import MTH5

from mt_metadata import TF_XML, TF_EDI_SPECTRA, TF_ZMM, TF_EDI_CGG
from mt_metadata.transfer_functions.core import TF

2022-03-17T21:22:44 [line 157] numexpr.utils._init_num_threads - INFO: NumExpr defaulting to 8 threads.


2022-03-17 21:22:47,367 [line 135] mth5.setup_logger - INFO: Logging file can be found C:\Users\jpeacock\Documents\GitHub\mth5\logs\mth5_debug.log


In [2]:
m = MTH5(file_version="0.2.0")
m.open_mth5(r"transfer_function_example.h5", "w")

2022-03-17 21:22:47,943 [line 601] mth5.mth5.MTH5.open_mth5 - WARNING: transfer_function_example.h5 will be overwritten in 'w' mode
2022-03-17 21:22:48,309 [line 660] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 file transfer_function_example.h5 in mode w


## Read in Transfer functions

* **TF_XML**: An example of EMTF XML format, the preferred format for archiving
* **TF_EDI_SPECTRA**: An example of an EDI file stored as spectra
* **TF_EDI_CGG**: An example of an output file from a contractor
* **TF_ZMM**: An example of an output file from EMFT


In [8]:
tf1 = TF(TF_XML)
tf2 = TF(TF_EDI_SPECTRA)
tf3 = TF(TF_EDI_CGG)
tf4 = TF(TF_ZMM)

## Add TF_XML to the MTH5

When we add a transfer function to an MTH5, it looks for the `survey.id` and `station.id`, if it doesn't find any then they are created.  If information is provided on which runs were processed and channels used those are filled in as well.

**Note:** If you have multiple transfer functions for a given station be sure to rename the file, for an EDI this is in `HEADER` under the attribute `DATAID`. Name it something like `station_sample_rate` or `station_runs` 

In [4]:
tf_group_01 = m.add_transfer_function(tf1)
tf_group_01

2022-03-17 21:22:49,315 [line 935] mth5.mth5.MTH5.get_survey - WARNING: /Experiment/Surveys/CONUS_South does not exist, check survey_list for existing names


/Experiment/Surveys/CONUS_South/Stations/NMX20/Transfer_Functions/NMX20:
    --> Dataset: inverse_signal_power
    ...................................
    --> Dataset: period
    .....................
    --> Dataset: residual_covariance
    ..................................
    --> Dataset: transfer_function
    ................................
    --> Dataset: transfer_function_error
    ......................................

### Have a look at what was added to the MTH5

Note that an EMTF XML has comprehensive metadata which can be used to populate the MTH5 as necessary, including `Runs` and `Channels`

In [5]:
m

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: CONUS_South
            ---------------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                    |- Group: fap
                    -------------
                    |- Group: fir
                    -------------
                    |- Group: time_delay
                    --------------------
                    |- Group: zpk
                    -------------
                |- Group: Reports
                -----------------
                |- Group: Standards
                -------------------
                    --> Dataset: summary
                    ............

## Add an example EDI

Here the survey is not specified therefore we need to fill that information in ourselves otherwise an error is raised, see below.

In [6]:
tf_group_02 = m.add_transfer_function(tf2)

AttributeError: 'NoneType' object has no attribute 'replace'

Here we give the survey the id `unknown_survey`.  Also note that because the data are stored as spectra in the EDI we can calculate the `inverse_signal_power` and `residual_covariance`.

In [17]:
tf2.survey_metadata.id = "unknown_survey"
tf_group_02 = m.add_transfer_function(tf2)
tf_group_02

/Experiment/Surveys/unknown_survey/Stations/SAGE_2005/Transfer_Functions/SAGE_2005:
    --> Dataset: inverse_signal_power
    ...................................
    --> Dataset: period
    .....................
    --> Dataset: residual_covariance
    ..................................
    --> Dataset: transfer_function
    ................................
    --> Dataset: transfer_function_error
    ......................................

## Add typical EDI file

This file only has impedance and tipper and minimal metadata, which are converted into a full transfer function for storage.

In [9]:
tf3.survey_metadata.id = "unknown_survey"
tf_group_03 = m.add_transfer_function(tf3)
tf_group_03

2022-03-17 21:27:22,495 [line 270] mth5.groups.base.TransferFunction.from_tf_object - WARNING: Did not find inverse_signal_power in TF. Skipping
2022-03-17 21:27:22,497 [line 270] mth5.groups.base.TransferFunction.from_tf_object - WARNING: Did not find residual_covariance in TF. Skipping


/Experiment/Surveys/unknown_survey/Stations/TEST01/Transfer_Functions/TEST01:
    --> Dataset: period
    .....................
    --> Dataset: transfer_function
    ................................
    --> Dataset: transfer_function_error
    ......................................

## Add an output from EMTF

A ZMM file contains the full covariance and transfer functions but has minimal metadata

In [11]:
tf4.survey_metadata.id = "unknown_survey"
tf_group_04 = m.add_transfer_function(tf4)
tf_group_04

/Experiment/Surveys/unknown_survey/Stations/300/Transfer_Functions/300:
    --> Dataset: inverse_signal_power
    ...................................
    --> Dataset: period
    .....................
    --> Dataset: residual_covariance
    ..................................
    --> Dataset: transfer_function
    ................................
    --> Dataset: transfer_function_error
    ......................................

## Have a look at the MTH5 file

Everything has been filled in now in the MTH5 including metadata about `runs` and `channels`

In [12]:
m

/:
    |- Group: Experiment
    --------------------
        |- Group: Reports
        -----------------
        |- Group: Standards
        -------------------
            --> Dataset: summary
            ......................
        |- Group: Surveys
        -----------------
            |- Group: CONUS_South
            ---------------------
                |- Group: Filters
                -----------------
                    |- Group: coefficient
                    ---------------------
                    |- Group: fap
                    -------------
                    |- Group: fir
                    -------------
                    |- Group: time_delay
                    --------------------
                    |- Group: zpk
                    -------------
                |- Group: Reports
                -----------------
                |- Group: Standards
                -------------------
                    --> Dataset: summary
                    ............

# Get a transfer function object from MTH5

To retrieve a transfer function from the MTH5 file a convenience function `m.get_transfer_function` is supplied.  You only need to know the `station.id`, `tf.id`, and the `survey.id`.  Here the `tf.id` is the same as the `station.id`.

In [13]:
tf1_h5 = m.get_transfer_function(tf1.station_metadata.id, tf1.station_metadata.id, tf1.survey_metadata.id)

In [14]:
print(tf1)
print(tf1_h5)

Station: NMX20
--------------------------------------------------
	Survey:            CONUS South
	Project:           USMTArray
	Acquired by:       National Geoelectromagnetic Facility
	Acquired date:     2020-09-20
	Latitude:          34.471
	Longitude:         -108.712
	Elevation:         1940.050
	Declination:   
		Value:     9.09
		Model:     WMM
	Coordinate System: geographic
	Impedance:         True
	Tipper:            True
	N Periods:     33
	Period Range:
		Min:   4.65455E+00 s
		Max:   2.91271E+04 s
	Frequency Range:
		Min:   3.43323E-05 Hz
		Max:   2.14844E-01 Hz
Station: NMX20
--------------------------------------------------
	Survey:            CONUS South
	Project:           USMTArray
	Acquired by:       National Geoelectromagnetic Facility
	Acquired date:     2020-09-20
	Latitude:          34.471
	Longitude:         -108.712
	Elevation:         1940.050
	Declination:   
		Value:     9.09
		Model:     WMM
	Coordinate System: geographic
	Impedance:         True
	Tipper:   

# MTpy

To analyze, plot, prepare input files one should look to use `MTpy`.  **Note:** `MTpy version 2.0` will use `MTH5` as the storage mechanism and `TF` to read/write files.